In [ ]:
import Pkg
Pkg.activate("../../../")

In [1]:
from laplace import Laplace

In [2]:
import pandas as pd
import torch

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(43)

# Load data from CSV file using pandas
df = pd.read_csv('data1.csv')

# Split the dataframe into x and y tensors
x = torch.from_numpy(df[['x1', 'x2']].to_numpy()).to(torch.float32)
y = torch.from_numpy(df['y'].to_numpy(dtype=int))

In [3]:
X = torch.tensor(x.T).float().T

# Convert y to a tensor of indices and one-hot encode it
y_unique = torch.unique(torch.tensor(y))
y_indices = torch.searchsorted(y_unique, torch.tensor(y))
y_train = nn.functional.one_hot(y_indices, num_classes=len(y_unique)).float()


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14096\1740726160.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(x.T).float().T
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14096\1740726160.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_unique = torch.unique(torch.tensor(y))
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14096\1740726160.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_indices = torch.searchsorted(y_unique, torch.tensor(y))


In [4]:
data = list(zip(x, y_train))
n_hidden = 3
D = X.shape[1]
out_dim = y_train.shape[1]
model = nn.Sequential(
    nn.Linear(D, n_hidden),
    nn.Sigmoid(),
    nn.Linear(n_hidden, out_dim)
)

In [5]:
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters())
epochs = 200
avg_loss = lambda data: torch.mean(torch.stack([loss_fn(model(x), y) for (x, y) in data]))
show_every = epochs // 10

In [6]:
for epoch in range(1, epochs+1):
    for (x, y) in data:
        opt.zero_grad()
        loss = loss_fn(model(x), y)
        loss.backward()
        opt.step()
    if epoch % show_every == 0:
        print("Epoch ", epoch)
        print("Avg Loss: ", avg_loss(data).item())

Epoch  20
Avg Loss:  0.661486029624939
Epoch  40
Avg Loss:  0.3299190402030945
Epoch  60
Avg Loss:  0.17765741050243378
Epoch  80
Avg Loss:  0.09842698276042938
Epoch  100
Avg Loss:  0.05515485629439354
Epoch  120
Avg Loss:  0.031066937372088432
Epoch  140
Avg Loss:  0.017541011795401573
Epoch  160
Avg Loss:  0.009913784451782703
Epoch  180
Avg Loss:  0.00560393463820219
Epoch  200
Avg Loss:  0.0031665244605392218


In [7]:
la = Laplace(model, 'classification',
             subset_of_weights='all',
             hessian_structure='full')

In [8]:
la.fit(DataLoader(TensorDataset(X, y_train), batch_size=len(y_train)))

In [9]:
la.optimize_prior_precision()

In [10]:
probit_predictions = la(X, link_approx='probit')

In [11]:
predictions_probit_df = pd.DataFrame(probit_predictions.numpy(), columns=['class1', 'class2', 'class3', 'class4'])

In [12]:
predictions_probit_df.to_csv('predictions1-Python.csv', index=False)

In [13]:
print(probit_predictions)

tensor([[0.7440, 0.0258, 0.1556, 0.0746],
        [0.7555, 0.0246, 0.1492, 0.0707],
        [0.7527, 0.0250, 0.1502, 0.0721],
        [0.7555, 0.0246, 0.1492, 0.0707],
        [0.7556, 0.0246, 0.1490, 0.0708],
        [0.7555, 0.0246, 0.1492, 0.0707],
        [0.7555, 0.0246, 0.1492, 0.0708],
        [0.7555, 0.0246, 0.1492, 0.0707],
        [0.7555, 0.0246, 0.1492, 0.0708],
        [0.7549, 0.0247, 0.1493, 0.0711],
        [0.7551, 0.0247, 0.1491, 0.0711],
        [0.4583, 0.0461, 0.2968, 0.1989],
        [0.7550, 0.0247, 0.1492, 0.0711],
        [0.7556, 0.0246, 0.1491, 0.0707],
        [0.7555, 0.0246, 0.1492, 0.0707],
        [0.7556, 0.0246, 0.1492, 0.0707],
        [0.7555, 0.0246, 0.1492, 0.0707],
        [0.7546, 0.0248, 0.1491, 0.0715],
        [0.7555, 0.0246, 0.1492, 0.0707],
        [0.7555, 0.0246, 0.1492, 0.0707],
        [0.7555, 0.0246, 0.1492, 0.0707],
        [0.7303, 0.0274, 0.1618, 0.0805],
        [0.6391, 0.0345, 0.2163, 0.1101],
        [0.7555, 0.0246, 0.1492, 0

In [14]:
torch.set_printoptions(sci_mode=False)
print(torch.softmax(model(X), dim=1))

tensor([[    0.9966,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9966,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9966,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9967,     0.0000,     0.0030,     0.0003],
        [    0.9966,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0

In [15]:
f_mu, f_var = la._glm_predictive_distribution(X)

In [16]:
print(f_mu)

tensor([[  4.4230,  -8.8740,  -1.2699,  -5.0352],
        [  4.4861,  -8.8812,  -1.1993,  -5.0710],
        [  4.4436,  -8.8760,  -1.2467,  -5.0467],
        [  4.4845,  -8.8810,  -1.2010,  -5.0701],
        [  4.4778,  -8.8802,  -1.2087,  -5.0662],
        [  4.4863,  -8.8812,  -1.1991,  -5.0711],
        [  4.4810,  -8.8800,  -1.2033,  -5.0695],
        [  4.4862,  -8.8812,  -1.1992,  -5.0710],
        [  4.4823,  -8.8804,  -1.2026,  -5.0696],
        [  4.4666,  -8.8789,  -1.2212,  -5.0599],
        [  4.4677,  -8.8790,  -1.2199,  -5.0605],
        [  3.5402,  -8.7660,  -2.2549,  -4.5300],
        [  4.4663,  -8.8789,  -1.2215,  -5.0597],
        [  4.4844,  -8.8810,  -1.2012,  -5.0700],
        [  4.4861,  -8.8812,  -1.1993,  -5.0710],
        [  4.4855,  -8.8811,  -1.2000,  -5.0706],
        [  4.4857,  -8.8811,  -1.1998,  -5.0707],
        [  4.4506,  -8.8739,  -1.2341,  -5.0530],
        [  4.4862,  -8.8812,  -1.1992,  -5.0710],
        [  4.4852,  -8.8809,  -1.1999,  -5.0709],


In [17]:
print(f_var.diagonal(dim1=1, dim2=2))

tensor([[ 31.0196,  41.0548,  31.7070,  52.5846],
        [ 28.6754,  41.1042,  28.9481,  52.8043],
        [ 29.1422,  41.0515,  29.4433,  52.4307],
        [ 28.6683,  41.1017,  28.9372,  52.7885],
        [ 28.6439,  41.0930,  28.8842,  52.6709],
        [ 28.6777,  41.1045,  28.9518,  52.8084],
        [ 28.6612,  41.0956,  28.9257,  52.7675],
        [ 28.6773,  41.1045,  28.9512,  52.8077],
        [ 28.6633,  41.0978,  28.9292,  52.7743],
        [ 28.7576,  41.0801,  28.9946,  52.5394],
        [ 28.7016,  41.0795,  28.9364,  52.5712],
        [416.1187,  45.3382, 513.8153, 164.9300],
        [ 28.7258,  41.0782,  28.9615,  52.5534],
        [ 28.6601,  41.1019,  28.9235,  52.7733],
        [ 28.6761,  41.1042,  28.9493,  52.8054],
        [ 28.6707,  41.1033,  28.9409,  52.7953],
        [ 28.6728,  41.1036,  28.9442,  52.7992],
        [ 28.7053,  41.0407,  28.9585,  52.5625],
        [ 28.6768,  41.1044,  28.9504,  52.8067],
        [ 28.6732,  41.1027,  28.9450,  52.7996],


In [18]:
torch.set_printoptions(sci_mode=False)
print(la.posterior_covariance)

tensor([[    33.2745,     -2.7266,      0.0084,      0.0185,     -0.0011,
             -0.0002,     -0.3882,      0.0053,     -0.0002,      0.0123,
              0.0913,     -0.1026,      0.1066,     -0.0531,      0.0336,
             -0.3959,      0.3983,     -0.3105,      0.2769,     -0.4366,
              0.3794,      0.0944,      0.0545,     -0.0592,     -0.0897],
        [    -2.7266,     23.2198,      0.0193,      0.1178,     -0.0019,
             -0.0011,     -1.8955,      0.0142,     -0.0007,     -0.0162,
              0.4702,     -0.2018,     -0.1011,      0.1467,      0.2128,
             -0.4680,      0.9486,     -0.6648,      0.5853,     -1.5656,
              0.6538,      0.3992,      0.0493,      0.2511,     -0.6998],
        [     0.0084,      0.0193,     34.0565,      0.0391,     -0.0002,
              0.0000,      0.0044,     -0.0083,     -0.0000,      0.0084,
             -0.0447,     -0.0501,     -0.0393,      0.0782,      0.0333,
              0.0301,     -0.0379,  

In [19]:
torch.set_printoptions(sci_mode=False)
print(la.H)

tensor([[     0.0011,      0.0038,     -0.0000,     -0.0000,     -0.0000,
             -0.0000,      0.0006,     -0.0000,      0.0000,     -0.0000,
              0.0002,      0.0000,      0.0000,      0.0000,      0.0000,
              0.0010,     -0.0008,      0.0011,     -0.0011,      0.0006,
             -0.0011,      0.0002,      0.0001,      0.0003,     -0.0005],
        [     0.0038,      0.0148,     -0.0000,     -0.0002,     -0.0000,
             -0.0000,      0.0026,     -0.0000,      0.0000,     -0.0001,
              0.0000,     -0.0000,     -0.0001,     -0.0001,     -0.0001,
              0.0016,     -0.0023,      0.0020,     -0.0014,      0.0024,
             -0.0019,      0.0000,     -0.0001,     -0.0005,      0.0006],
        [    -0.0000,     -0.0000,      0.0000,     -0.0000,     -0.0000,
             -0.0000,     -0.0000,      0.0000,     -0.0000,     -0.0000,
              0.0002,      0.0002,      0.0000,     -0.0002,     -0.0002,
             -0.0000,     -0.0000,  

In [20]:
print(torch.diag(la.prior_precision_diag))

tensor([[0.0294, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0294, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0294, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0294, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0294, 0.0000, 0.0000, 0.0000, 0.0000,
       

In [21]:
torch.set_printoptions(sci_mode=False)
Js, f_mu = la.backend.jacobians(X)
Js = Js.permute(2, 0, 1).contiguous().view(25, -1)
for i in range(6):
    print(Js[i, :3])

print()
for i in range(6, 9):
    print(Js[i, :3])
print()
for i in range(9, 21):
    print(Js[i, :3])
print()
for i in range(21, 25):
    print(Js[i, :3])

tensor([-0.1412,  0.0156, -0.1582])
tensor([-0.3334,  0.0370, -0.3735])
tensor([     0.0002,     -0.0005,      0.0000])
tensor([     0.0005,     -0.0013,      0.0001])
tensor([    -0.0000,      0.0000,      0.0000])
tensor([    -0.0000,      0.0000,      0.0000])

tensor([-0.0625,  0.0069, -0.0700])
tensor([     0.0001,     -0.0002,      0.0000])
tensor([    -0.0000,      0.0000,      0.0000])

tensor([0.0118, 0.0000, 0.0000])
tensor([1.0000, 0.0000, 0.0000])
tensor([    0.0000,     0.0000,     0.0000])
tensor([0.0000, 0.0118, 0.0000])
tensor([0.0000, 1.0000, 0.0000])
tensor([    0.0000,     0.0000,     0.0000])
tensor([0.0000, 0.0000, 0.0118])
tensor([0.0000, 0.0000, 1.0000])
tensor([    0.0000,     0.0000,     0.0000])
tensor([0., 0., 0.])
tensor([0., 0., 0.])
tensor([0., 0., 0.])

tensor([1., 0., 0.])
tensor([0., 1., 0.])
tensor([0., 0., 1.])
tensor([0., 0., 0.])
